# Experiment: 04 Nonlinear Activation Momentum

Compare activation functions on EMA momentum signals.

In [1]:

import os
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import shutil

warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True, linewidth=140)

BASE_DIR = Path("/Users/kencharoff/workspace/projects/momentum/research")
INPUT_CSV = Path("/Users/kencharoff/workspace/projects/momentum/research/data/prices.csv")

OUTPUT_DIR = BASE_DIR / "outputs"
LOG_DIR = OUTPUT_DIR / "logs"
PLOTS_DIR = OUTPUT_DIR / "plots"
TABLES_DIR = OUTPUT_DIR / "tables"
PERF_DIR = OUTPUT_DIR / "perf"

for d in [LOG_DIR, PLOTS_DIR, TABLES_DIR, PERF_DIR]:
    d.mkdir(parents=True, exist_ok=True)

CONFIG = {
    "input_csv": str(INPUT_CSV),
    "universe_min_coverage": 0.7,
    "rebalance": "ME",
    "cost_bps": 5.0,
    "mom_lookback_m": 12,
    "mom_skip_m": 1,
    "mom_form_m": 6,
    "beta_window_d": 252,
    "cov_lookback_m": 36,
    "ppp_eta": 2.0,
    "ppp_gamma": 10.0,
    "ppp_winner_pct": 0.3,
    "granger_window_m": 36,
    "granger_lags": 2,
    "bootstrap_B": 500,
    "activation_span": 12,
    "deadzone_eps_grid": [0.0,0.2,0.4,0.6,0.8,1.0],
    "trend_horizons_m": [1,3,12],
    "vol_ewma_span": 63,
    "dd_stop": 0.2,
    "stop_scale": 0.5,
    "k_top_bottom": 3,
    "comom_window_m": 36,
}

def backup_if_exists(path: Path):
    path = Path(path)
    if path.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = path.with_name(path.name + f".bak.{ts}")
        shutil.copy2(path, bak)
        return bak
    return None

def safe_write(path: Path, writer_fn):
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    backup_if_exists(path)
    writer_fn(path)

def normalize_field(x: str) -> str:
    s = str(x).strip().lower().replace(" ", "_")
    if s in {"adj_close", "adjclose", "adjusted_close", "adjustedclose"}:
        return "adj_close"
    if s in {"close", "open", "high", "low", "volume", "price"}:
        return s
    return s

def load_prices_csv(path: Path):
    df = pd.read_csv(path, header=None)
    row0 = df.iloc[0].astype(str).fillna("")
    row1 = df.iloc[1].astype(str).fillna("")

    date_label_rows = df.index[df.iloc[:, 0].astype(str).str.strip().str.lower() == "date"]
    date_label_idx = int(date_label_rows[0]) if len(date_label_rows) > 0 else 2
    data_start_idx = date_label_idx + 1

    def count_fields(row):
        field_set = {"adj close", "close", "open", "high", "low", "volume", "price"}
        return sum(str(v).strip().lower() in field_set for v in row)

    if row1.iloc[0].strip().lower() == "ticker":
        field_row, ticker_row = row0, row1
    elif row0.iloc[0].strip().lower() == "ticker":
        field_row, ticker_row = row1, row0
    else:
        if count_fields(row0) >= count_fields(row1):
            field_row, ticker_row = row0, row1
        else:
            field_row, ticker_row = row1, row0

    dates = pd.to_datetime(df.iloc[data_start_idx:, 0], errors="coerce")
    data = df.iloc[data_start_idx:, 1:].copy()

    cols = []
    for j in range(1, df.shape[1]):
        field = normalize_field(field_row.iloc[j])
        ticker = str(ticker_row.iloc[j]).strip()
        if ticker in ("", "nan", "None"):
            cols.append((field, None))
        else:
            cols.append((field, ticker))
    data.columns = pd.MultiIndex.from_tuples(cols)
    data = data.loc[:, [c for c in data.columns if c[0] and c[1]]]
    data = data.apply(pd.to_numeric, errors="coerce")
    data.index = dates
    data = data[~data.index.isna()]

    panel = {}
    for field in sorted(set([c[0] for c in data.columns])):
        panel[field] = data.xs(field, level=0, axis=1).copy()
    return panel

def to_month_end(px_d: pd.DataFrame) -> pd.DataFrame:
    return px_d.resample("ME").last()

def returns_from_prices(px: pd.DataFrame) -> pd.DataFrame:
    return px.pct_change()

def summarize_panel(adj_close: pd.DataFrame) -> pd.DataFrame:
    coverage = adj_close.notna().mean()
    start = adj_close.apply(lambda s: s.first_valid_index())
    end = adj_close.apply(lambda s: s.last_valid_index())
    summary = pd.DataFrame({"coverage": coverage, "start": start, "end": end})
    return summary.sort_values("coverage")

def filter_universe_by_coverage(adj_close: pd.DataFrame, min_coverage: float):
    coverage = adj_close.notna().mean()
    keep = coverage >= min_coverage
    excluded = coverage[~keep].sort_values()
    return adj_close.loc[:, keep], excluded

def map_rebalance_to_next_trading_day(rebalance_dates, trading_dates):
    rebalance_dates = pd.DatetimeIndex(rebalance_dates).sort_values()
    trading_dates = pd.DatetimeIndex(trading_dates).sort_values()
    apply_dates = []
    for t in rebalance_dates:
        idx = trading_dates.searchsorted(t, side="right")
        if idx >= len(trading_dates):
            apply_dates.append(pd.NaT)
        else:
            apply_dates.append(trading_dates[idx])
    rows = []
    for i, t in enumerate(rebalance_dates):
        start = apply_dates[i]
        if pd.isna(start):
            end = pd.NaT
        else:
            if i + 1 < len(apply_dates) and pd.notna(apply_dates[i + 1]):
                end = trading_dates[trading_dates < apply_dates[i + 1]][-1]
            else:
                end = trading_dates[-1]
        rows.append({"rebalance_date": t, "apply_start": start, "apply_end": end})
    return pd.DataFrame(rows)

def expand_weights_monthly_to_daily(w_target: pd.DataFrame, trading_dates):
    w_target = w_target.copy()
    w_target.index = pd.DatetimeIndex(w_target.index)
    mapping = map_rebalance_to_next_trading_day(w_target.index, trading_dates)
    trading_dates = pd.DatetimeIndex(trading_dates).sort_values()
    w_applied = pd.DataFrame(index=trading_dates, columns=w_target.columns, dtype=float)
    for row in mapping.itertuples(index=False):
        if pd.isna(row.apply_start):
            continue
        mask = (w_applied.index >= row.apply_start) & (w_applied.index <= row.apply_end)
        w_applied.loc[mask] = w_target.loc[row.rebalance_date].values
    w_applied = w_applied.ffill().fillna(0.0)
    return w_applied, mapping

def apply_portfolio(ret_d: pd.DataFrame, w_target: pd.DataFrame, cost_bps: float = 0.0):
    w_target = w_target.reindex(columns=ret_d.columns).fillna(0.0)
    w_applied, mapping = expand_weights_monthly_to_daily(w_target, ret_d.index)
    w_prev = w_applied.shift(1).fillna(0.0)
    r_p = (w_prev * ret_d).sum(axis=1)
    turnover = w_applied.diff().abs().sum(axis=1).fillna(0.0)
    cost = (cost_bps / 1e4) * turnover
    ret_net = r_p - cost
    nav = (1.0 + ret_net).cumprod()
    return {
        "nav": nav,
        "ret_net": ret_net,
        "turnover": turnover,
        "cost": cost,
        "w_applied_d": w_applied,
        "mapping": mapping,
    }

def metrics(nav: pd.Series, ret: pd.Series, periods_per_year: int):
    ret = ret.dropna()
    if len(ret) < 2:
        return {
            "cagr": np.nan,
            "vol": np.nan,
            "sharpe": np.nan,
            "maxdd": np.nan,
            "calmar": np.nan,
            "skew": np.nan,
            "kurt": np.nan,
        }
    nav = nav.loc[ret.index]
    total_years = len(ret) / periods_per_year
    cagr = (nav.iloc[-1] / nav.iloc[0]) ** (1 / total_years) - 1 if total_years > 0 else np.nan
    vol = ret.std() * np.sqrt(periods_per_year)
    sharpe = ret.mean() / ret.std() * np.sqrt(periods_per_year) if ret.std() > 0 else np.nan
    dd = nav / nav.cummax() - 1.0
    maxdd = dd.min()
    calmar = -cagr / maxdd if maxdd < 0 else np.nan
    return {
        "cagr": cagr,
        "vol": vol,
        "sharpe": sharpe,
        "maxdd": maxdd,
        "calmar": calmar,
        "skew": ret.skew(),
        "kurt": ret.kurtosis(),
    }

def write_log(path: Path, lines):
    def _writer(p):
        with open(p, "w", encoding="utf-8") as f:
            f.write("\n".join(lines) + "\n")
    safe_write(path, _writer)

def save_fig(path: Path):
    def _writer(p):
        plt.savefig(p, dpi=150, bbox_inches="tight")
    safe_write(path, _writer)


In [2]:

panel_path = TABLES_DIR / "panel.pkl"
if panel_path.exists():
    panel = pd.read_pickle(panel_path)
else:
    panel = load_prices_csv(INPUT_CSV)

if "adj_close" not in panel:
    raise ValueError("adj_close not found in panel")

adj_close_raw = panel["adj_close"].copy()
summary = summarize_panel(adj_close_raw)
adj_close, excluded = filter_universe_by_coverage(adj_close_raw, CONFIG["universe_min_coverage"])
panel["adj_close"] = adj_close

ret_d = returns_from_prices(adj_close).fillna(0.0)
px_m = to_month_end(adj_close)
ret_m = returns_from_prices(px_m)
mapping = map_rebalance_to_next_trading_day(px_m.index, adj_close.index)


In [3]:

# Monthly EMA momentum signal
ret_m = returns_from_prices(px_m)
signal = ret_m.ewm(span=CONFIG["activation_span"], min_periods=3).mean()

def normalize_longshort(sig_row):
    s = sig_row.fillna(0.0)
    denom = s.abs().sum()
    if denom == 0:
        return pd.Series(0.0, index=s.index)
    return s / denom

def activation_linear(x):
    return x

def activation_sign(x):
    return np.sign(x)

def activation_deadzone(x, eps):
    ax = np.abs(x)
    return np.where(ax < eps, 0.0, np.sign(x) * (ax - eps))

def activation_tanh(x, k=2.0):
    return np.tanh(k * x)

activations = {
    "linear": lambda x: activation_linear(x),
    "sign": lambda x: activation_sign(x),
    "deadzone_0.2": lambda x: activation_deadzone(x, 0.2),
    "tanh": lambda x: activation_tanh(x, 2.0),
}

results = {}
for name, fn in activations.items():
    w_list = []
    for t in signal.index:
        sig_row = pd.Series(fn(signal.loc[t].values), index=signal.columns)
        w_list.append(normalize_longshort(sig_row))
    w_target = pd.DataFrame(w_list, index=signal.index, columns=signal.columns)
    res = apply_portfolio(ret_d, w_target, CONFIG["cost_bps"])
    results[name] = res

# Metrics
rows = []
for name, res in results.items():
    m = metrics(res["nav"], res["ret_net"], periods_per_year=252)
    m["strategy"] = name
    rows.append(m)
metrics_df = pd.DataFrame(rows)
safe_write(TABLES_DIR / "activation_metrics.csv", lambda p: metrics_df.to_csv(p, index=False))

# Deadzone sweep
sharpe_list = []
for eps in CONFIG["deadzone_eps_grid"]:
    w_list = []
    for t in signal.index:
        sig_row = pd.Series(activation_deadzone(signal.loc[t].values, eps), index=signal.columns)
        w_list.append(normalize_longshort(sig_row))
    w_target = pd.DataFrame(w_list, index=signal.index, columns=signal.columns)
    res = apply_portfolio(ret_d, w_target, CONFIG["cost_bps"])
    m = metrics(res["nav"], res["ret_net"], periods_per_year=252)
    sharpe_list.append(m["sharpe"])

plt.figure(figsize=(6, 4))
plt.plot(CONFIG["deadzone_eps_grid"], sharpe_list, marker="o")
plt.title("Deadzone sweep (Sharpe)")
plt.xlabel("deadzone eps")
plt.ylabel("Sharpe")
plt.grid(True, alpha=0.3)
save_fig(PLOTS_DIR / "deadzone_sweep.png")
plt.close()

log_lines = []
log_lines.append(f"input_csv: {INPUT_CSV}")
log_lines.append(f"tickers_filtered: {adj_close.shape[1]}")
log_lines.append(f"date_range: {adj_close.index.min().date()} to {adj_close.index.max().date()}")
log_lines.append("\nexcluded_tickers (coverage < min):")
log_lines.append(excluded.to_string())
log_lines.append("\nmonth_end_to_next_trading_day:")
log_lines.append(mapping.to_string(index=False))
log_lines.append("\noutputs:")
log_lines.append(str(TABLES_DIR / "activation_metrics.csv"))
log_lines.append(str(PLOTS_DIR / "deadzone_sweep.png"))
write_log(LOG_DIR / "04_activation.md", log_lines)
